In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from MTGpred.utils.database import get_all_decks,get_all_tournaments

db_decks = get_all_decks()

cards_distribution = {}

for deck in db_decks:
    for card in deck["main_deck"]:
        if card["name"] in cards_distribution:
            cards_distribution[card["name"]] += int(card["quantity"])
        else:
            cards_distribution[card["name"]] = int(card["quantity"])

In [3]:
sorted_cards_distribution = {k: v for k, v in sorted(cards_distribution.items(), key=lambda item: -int(item[1]))}

In [4]:
db_tournaments = get_all_tournaments()

formats_distribution = {}

for tournament in db_tournaments:
    if tournament["format"] in formats_distribution:
        formats_distribution[tournament["format"]] += 7
    else:
        formats_distribution[tournament["format"]] = 7

In [5]:
print(formats_distribution)

{'modern': 424, 'sealed': 285, 'standard': 315, 'pauper': 320, 'legacy': 267, 'pioneer': 221, 'other': 348, 'commander': 57, 'draft': 5}


In [11]:
from MTGpred.utils.database import get_matches_id_by_format,get_match
from MTGpred.model.dataset import MatchesDataset3
from MTGpred.utils.mtgjson import load_cards_df
from MTGpred.model.transformer import WinnerPredictor
import torch

matches = get_matches_id_by_format("standard")
cards_df = load_cards_df("../data/AtomicCards.json")
dataset = MatchesDataset3(cards_df,matches,random_output=False)
model = WinnerPredictor()
model.load_state_dict(torch.load("../models/transformer.pt"))
model.eval()

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassifica

WinnerPredictor(
  (encoder): BigBirdForSequenceClassification(
    (bert): BigBirdModel(
      (embeddings): BigBirdEmbeddings(
        (word_embeddings): Embedding(50358, 768, padding_idx=0)
        (position_embeddings): Embedding(4096, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BigBirdEncoder(
        (layer): ModuleList(
          (0): BigBirdLayer(
            (attention): BigBirdAttention(
              (self): BigBirdBlockSparseAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
              )
              (output): BigBirdSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (L

In [12]:
def create_batch(batch_encoding):
    batch = {}
    for key in batch_encoding:
        batch[key] = batch_encoding[key].unsqueeze(0)
    return batch

for i,(deck1,deck2,target) in enumerate(dataset):
    deck1 = create_batch(deck1)
    deck2 = create_batch(deck2)

    match_id = dataset.get_match_id(i)
    match_data = get_match(match_id)
    print()
    print(match_data["p1_name"],match_data["p1_points"],"-",match_data["p2_name"],match_data["p2_points"])
    print(model(deck1,deck2),target)
    print(model(deck2,deck1))
    
    if i > 5:
        break


Edel 2 - stainerson 1


/home/data/ucadatalab_group/envs/mtg-preds/lib/python3.9/site-packages/transformers/models/big_bird/modeling_big_bird.py:980: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  torch.arange(indices.shape[0] * indices.shape[1] * num_indices_to_gather, device=indices.device)


tensor(0.1028, grad_fn=<SqueezeBackward0>) 0.5
tensor(0.0455, grad_fn=<SqueezeBackward0>)

Brunister 2 - _Batutinha_ 0
tensor(-0.0084, grad_fn=<SqueezeBackward0>) 1.0
tensor(0.1289, grad_fn=<SqueezeBackward0>)

bolov0 2 - stonepeanut 1
tensor(-0.0397, grad_fn=<SqueezeBackward0>) 0.5
tensor(-0.0439, grad_fn=<SqueezeBackward0>)

Venom1 2 - Morenothings 1
tensor(0.1443, grad_fn=<SqueezeBackward0>) 0.5
tensor(0.0069, grad_fn=<SqueezeBackward0>)

Edel 2 - Brunister 0
tensor(0.1028, grad_fn=<SqueezeBackward0>) 1.0
tensor(0.0455, grad_fn=<SqueezeBackward0>)

bolov0 2 - Venom1 0
tensor(0.0351, grad_fn=<SqueezeBackward0>) 1.0
tensor(0.0637, grad_fn=<SqueezeBackward0>)

Edel 2 - bolov0 0
tensor(0.0248, grad_fn=<SqueezeBackward0>) 1.0
tensor(0.0597, grad_fn=<SqueezeBackward0>)
